In [20]:
import pandas as pd
import re
import itertools

In [21]:
# Define import functions
def import_and_clean(path):
    file = pd.read_json(path)
    data = pd.DataFrame(list(itertools.chain(*file['message'].apply(lambda x: x['items']))))
    data['earliest_pub_year'] = data['published'].apply(lambda x: x['date-parts'][0][0])
    data['cleaned_abstract'] = data['abstract'].apply(lambda x: re.sub("<.*?>|\n|Abstract|abstract", "", str(x)))
    return data

In [22]:
climate_articles_2013 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2013.json')
climate_articles_2014 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2014.json')
climate_articles_2015 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2015.json')
climate_articles_2016 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2016.json')
climate_articles_2017 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2017.json')
climate_articles_2018 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2018.json')
climate_articles_2019 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2019.json')
climate_articles_2020 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2020.json')
climate_articles_2021 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2021.json')
climate_articles_2022 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2022.json')
climate_articles_2023 = import_and_clean('/Users/trevor/Desktop/Research/climate-trends/climate_articles_2023.json')

In [23]:
climate_articles = pd.concat([
    climate_articles_2013,
    climate_articles_2014,
    climate_articles_2015,
    climate_articles_2016,
    climate_articles_2017,
    climate_articles_2018,
    climate_articles_2019,
    climate_articles_2020,
    climate_articles_2021,
    climate_articles_2022,
    climate_articles_2023
])

In [24]:
climate_articles.shape
# 284,231 articles extracted- still need to clean/ check for potential DOI duplicates

(284231, 56)